In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Linear Regression Examplt").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 20:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv("Ecommerce_Customers.csv", header=True, inferSchema=True)
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [7]:
assembler = VectorAssembler(
    inputCols=[
        "Avg Session Length",
        "Time on App",
        "Time on Website",
        "Length of Membership",
    ],
    outputCol="features",
)

24/03/09 20:55:48 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
output = assembler.transform(data)

In [9]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [11]:
final_data = output.select("features", "Yearly Amount Spent")

In [13]:
final_data.show(truncate=False)

+-----------------------------------------------------------------------------+-------------------+
|features                                                                     |Yearly Amount Spent|
+-----------------------------------------------------------------------------+-------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]   |587.9510539684005  |
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]   |392.2049334443264  |
|[33.000914755642675,11.330278057777512,37.110597442120856,4.104543202376424] |487.54750486747207 |
|[34.30555662975554,13.717513665142507,36.72128267790313,3.120178782748092]   |581.8523440352177  |
|[33.33067252364639,12.795188551078114,37.53665330059473,4.446308318351434]   |599.4060920457634  |
|[33.871037879341976,12.026925339755056,34.47687762925054,5.493507201364199]  |637.102447915074   |
|[32.02159550138701,11.366348309710526,36.68377615286961,4.685017246570912]   |521.5721747578274  |


In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean| 501.61776678170935|
| stddev|  80.18428529786127|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean|  493.9386717046468|
| stddev|  77.24416163484982|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [18]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [19]:
lr_model = lr.fit(train_data)

24/03/09 21:43:26 WARN Instrumentation: [ade9851c] regParam is zero, which might cause numerical instability and overfitting.
24/03/09 21:43:27 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/09 21:43:27 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/03/09 21:43:27 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [20]:
test_resutl = lr_model.evaluate(test_data)

In [23]:
test_resutl.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.204560029286824|
|-0.5694771823917222|
|-6.7223580362727375|
|-22.034649545012257|
| -4.775620405291647|
| -4.512654657383848|
|  -18.3482585908273|
|-2.7074187342976757|
| -4.771372526238622|
| -5.375180030658214|
|-2.3081782029534565|
|  7.254704762898143|
|  8.070256022250817|
| -5.887757942911037|
|-1.8589665396868327|
| 11.788211836460107|
|  7.992149838522778|
|   5.43879326479032|
| -8.921034977737008|
|  4.690072859813995|
+-------------------+
only showing top 20 rows



In [24]:
test_resutl.rootMeanSquaredError

9.35246699337282

In [25]:
test_resutl.r2

0.9852420501202652

In [26]:
unlabeled_data = test_data.select("features")

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[31.0613251567161...|
|[31.1239743499119...|
|[31.4252268808548...|
|[31.5171218025062...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6253601348306...|
|[31.7656188210424...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9549038566348...|
|[31.9764800614612...|
|[32.0085045178551...|
|[32.0478009788678...|
+--------------------+
only showing top 20 rows



In [28]:
prediction = lr_model.transform(unlabeled_data)

In [29]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|287.67580574920134|
|[30.5743636841713...| 442.6338909404574|
|[31.0613251567161...|494.27781609417434|
|[31.1239743499119...|508.98170338477803|
|[31.4252268808548...| 535.5423390600536|
|[31.5171218025062...|280.43107530776956|
|[31.5702008293202...| 564.2937507322322|
|[31.5761319713222...|  543.934002723626|
|[31.6253601348306...| 381.1082732831628|
|[31.7656188210424...|501.92926166626535|
|[31.8186165667690...| 448.7268515730891|
|[31.8209982016720...| 417.4205762503152|
|[31.8512531286083...| 464.9219906445476|
|[31.8745516945853...|398.17300218917853|
|[31.9120759292006...|389.39368284539455|
|[31.9262720263601...| 380.4167216078663|
|[31.9549038566348...| 432.0057301014042|
|[31.9764800614612...| 325.1556527693099|
|[32.0085045178551...| 452.1182560064924|
|[32.0478009788678...|508.76049832628246|
+--------------------+------------